In [1]:
cd ../..

/home/mb/MEGAsync/academic_work/projects/geometric_model_polycrystals/code/PyAPD


In [2]:
# installing the prototype local version of `PyAPD`
!pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/mb/MEGAsync/academic_work/projects/geometric_model_polycrystals/code/PyAPD
  Preparing metadata (setup.py) ... done
  Attempting uninstall: PyAPD
    Found existing installation: PyAPD 0.1.1
    Uninstalling PyAPD-0.1.1:
      Successfully uninstalled PyAPD-0.1.1
  Running setup.py develop for PyAPD


In [23]:
import PyAPD
import time
from pykeops.torch import LazyTensor
import torch
from torchmin import minimize as minimize_torch
import copy
#import ipywidgets as widgets
from matplotlib import animation
from matplotlib import pyplot as plt

from pysdot.domain_types import ConvexPolyhedraAssembly
#from pysdot.domain_types import ScaledImage
from pysdot import OptimalTransport
from pysdot import PowerDiagram

In [32]:
N = 10000
apd = PyAPD.apd_system(N=N,
                       ani_thres=0.05,
                       pixel_size_prefactor=2,
                       dt=torch.float64,
                       seed = 30,
                      error_tolerance=0.01)
# apd.check_optimality()
W0 = apd.W
# apd.find_optimal_W()

In [30]:
# apd.check_optimality()

We would like to be able to compute optimal $W$ faster than this. Putting aside the issues of hardware, this can be achieved if we somehow lower the number of total iterations/function evaluations. 

To this end we would like to utilise the isotropic case and `pysdot` in particular. A prototype of `pysdot` and `PyAPD` integration is as follows. 

In [33]:
st=time.time()
dd = apd.domain.numpy().transpose()
dm=ConvexPolyhedraAssembly(); # Create a polyhedral assembly
dm.add_box(dd[0],dd[1])
ot=OptimalTransport(positions=apd.X.numpy(),
                    weights=W0.numpy(),
                    masses=apd.target_masses.numpy(),
                    domain=dm,
                    verbosity=1)
ot.set_stopping_criterion(apd.target_masses[0]*apd.error_tolerance,
                          type="max delta masses")
ot.adjust_weights()
en=time.time()
print('Elapsed time {}'.format(en-st))

Sucessfully imported sparse linear solver Scipy.
Elapsed time 1.2574055194854736


We can now pass weights from `pysdot` to `PyAPD` and then use numerical continuation to gradually introduce anisotropy. 

The idea of numerical continuation is to introduce a parameter $t \in [0,1]$ and have each anisotropy matrix parametrised by it, $A_i^t$ such that $A^0_i = {\rm Id}$ and $A^1_i = A_i$. The initial prototyping is done for linear blending, so
$$
A^t_i = (1-t){\rm Id} + tA_i,
$$
but this creates a mild issue that ${\rm det} A_t^i \neq 1$. At least in 2D a simple alternative to be perhaps explored is to have $A_i^t = V_iD_i^tV_i^{-1}$ with $D_i = {\rm diag}(a_i^t, 1/a_i^t)$ where $a_i^t = (1-t) + ta_i$. 


Clearly when $t=0$, we have the isotropic problem. Let us pass the computed weights and with new helper functions first check if it solves the pixelated isotropic problem.

In [7]:
apd.set_W(torch.tensor(ot.get_weights()))
apd.from_t_to_W(0.0)

Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0005)
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.022779
         Iterations: 1
         Function evaluations: 3
It took 0.007108211517333984 seconds to find optimal W.


We can then simply use this as an initial guess and solve the problem directly:

In [8]:
apd.set_W(torch.tensor(ot.get_weights()))
apd.find_optimal_W()

Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0005)
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.023020
         Iterations: 6
         Function evaluations: 8
It took 0.02806997299194336 seconds to find optimal W.


The alternative is to try some sort of continuation routine as follows:

In [9]:
apd.set_W(torch.tensor(ot.get_weights()))
start=time.time()
apd.continuation_routine(steps=10,
                         tangent_info=True,
                        error_prefactor = 1.0,
                        verbose = True)
time_taken = time.time()-start
time_taken

Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0005)
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.022779
         Iterations: 1
         Function evaluations: 3
It took 0.011475324630737305 seconds to find optimal W.
Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0005)
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.022813
         Iterations: 2
         Function evaluations: 4
It took 0.016025543212890625 seconds to find optimal W.
Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0005)
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.022845
         Iterations: 2
         Function evaluations: 4
It took 0.01102137565612793 seconds to find optimal W.
Solver toleran

0.1342296600341797

The simple continuation routine where the new initial guess to find $W^{i+1}$ is just $2W^{i} - W^{i-1}$ (simple tangent information with fixed step size) is just not good enough in any combination I have tested it for. 

### "Annealing" of error tolerance (and hence pixels)

In [18]:
N = 300
apd = PyAPD.apd_system(N=N,
                       ani_thres=0.5,
                       pixel_size_prefactor=2,
                       dt=torch.float64,
                       seed = 40,
                      error_tolerance=0.01)
apd.check_optimality()
W0 = apd.W
apd.find_optimal_W()
apd.check_optimality()

Precision loss detected!
Percentage error =  158.11086237429046
Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(3.3333e-05)
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.002266
         Iterations: 92
         Function evaluations: 98
It took 10.033132553100586 seconds to find optimal W.
The APD is optimal!
Percentage error =  0.7651441745472071


In [19]:

# start with large error tolerance (so much fewer pixels)
# and then slowly decrease to a small value
errs = [0.5,0.25,0.125,0.05,0.025,0.0125,0.01]

N = 300
W_restart = torch.zeros(N)
start=time.time()
for err in errs:
    apd = PyAPD.apd_system(N=N,
                           ani_thres=0.5,
                           pixel_size_prefactor=2,
                           dt=torch.float64,
                           seed = 40,
                          error_tolerance=err)
    #apd.pixel_params = (282,282)
    apd.check_optimality()
    print(apd.pixel_params)
    apd.set_W(W=W_restart)
    apd.find_optimal_W()
    apd.check_optimality()
    W_restart = apd.W

time_taken = time.time()-start
time_taken

Precision loss detected!
Percentage error =  160.41666666666669
(48, 48)
Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0017)
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.002263
         Iterations: 44
         Function evaluations: 47
It took 0.3005216121673584 seconds to find optimal W.
The APD is optimal!
Percentage error =  47.91666666666667
Precision loss detected!
Percentage error =  166.0034602076123
(68, 68)
Solver tolerance is with respect to each grain separately.
Smallest tol:  tensor(0.0008)
Optimality condition successfully overwritten.
Optimization terminated successfully.
         Current function value: -0.002263
         Iterations: 4
         Function evaluations: 6
It took 0.03354310989379883 seconds to find optimal W.
The APD is optimal!
Percentage error =  23.26989619377158
Precision loss detected!
Percentage error =  163.67187499999952
(96, 96)
Solver to

7.1142261028289795